In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from hazm import *
import re
from string import punctuation
import pandas as pd
from cleantext import clean


In [2]:
def delete_stop_words(string):
    cleaned_string = re.sub(f'[{punctuation}؟،٪×÷»«]+', '', string)
    stop_words = utils.stopwords_list()
    tokenizer = WordTokenizer()
    lst = tokenizer.tokenize(cleaned_string)
    res = []
    for token in lst:
        if token not in stop_words:
            res.append(token)
    result = " ".join(res)
    return result

def cleaning(text):
    text = text.strip()

    text = clean(text,
    fix_unicode=True,
    to_ascii=False,
    lower=True,
    no_line_breaks=True,
    no_urls=True,
    no_emails=True,
    no_phone_numbers=True,
    no_numbers=False,
    no_digits=False,
    no_currency_symbols=True,
    no_punct=False,
    replace_with_url="",
    replace_with_email="",
    replace_with_phone_number="",
    replace_with_number="",
    replace_with_digit="0",
    replace_with_currency_symbol="",
    )

    normalizer = Normalizer()
    text = normalizer.normalize(text)
    wierd_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               u"\u2069"
                               u"\u2066"
                               u"\u200c"
                               u"\u2068"
                               u"\u2067"
                               "]+", flags=re.UNICODE)

    text = wierd_pattern.sub(r'', text)
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)
    text = normalizer.affix_spacing(text)
    text = normalizer.character_refinement(text)
    text = normalizer.punctuation_spacing(text)
    return text

In [3]:
# g = open("train.txt", "w+")
# h = open("test.txt", "w+")



persica = PersicaReader('persica.csv')
i = 0
docs = persica.docs()
obj = next(docs)

Columns = ['text', 'label']
train_df = pd.DataFrame(columns=Columns)
test_df = pd.DataFrame(columns=Columns)
df = pd.DataFrame(columns=Columns)
i = 0
print(obj.keys())
while obj:
    text = obj['text']
    label = obj['category2']
    text = cleaning(text)
    df = pd.concat([df, pd.DataFrame.from_records([{'text': text, 'label': label}])])
    try:
         obj = next(docs)
         i += 1
    except StopIteration:
         break
df.reset_index(drop=True)
# print(df.)
# df = df.drop(columns=['index'])
# g.close()
# h.close()

dict_keys(['id', 'title', 'text', 'date', 'time', 'category', 'category2'])


,text,label
0,وزیر علوم در جمع استادان نمونه کشور گفت: از اس...,آموزشي
1,به گزارش سرویس صنفی آموزشی خبرگزاری دانشجویان ...,آموزشي
2,نتایج آزمون دورههای فراگیر مقاطع کارشناسی و کا...,آموزشي
3,,اجتماعي
4,محمدتقی علوی یزدی، مجری این طرح پژوهشی در اینب...,آموزشي
...,...,...
10994,به گزارش ایسنا، در کوران مسابقات دوچرخه سواری ...,ورزشي
10995,محمد دلیریان در گفتوگو با خبرنگار کشتی خبرگزار...,ورزشي
10996,به گزارش خبرگزاری دانشجویان ایران (ایسنا)، در ...,ورزشي
10997,به گزارش خبرگزاری دانشجویان ایران (ایسنا)، ایر...,ورزشي


In [4]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

df['target'] = le.fit_transform(df['label'])
df['label'].unique()

df = df.sample(frac=1)

train_df = df.sample(frac=0.8, ignore_index=True)
test_df = pd.concat([df, train_df])
test_df = test_df.drop_duplicates(ignore_index=True, keep=False)

In [5]:
test_df['label'].nunique()

11

In [6]:
train_df['label'].nunique()

11

In [7]:
test_df['target'].iloc[0:50]

0      1
1      5
2      3
3      4
4      4
5      2
6      9
7     10
8      1
9      0
10    10
11     9
12     6
13     3
14     6
15     3
16     3
17     7
18     2
19     2
20     2
21    10
22     6
23     1
24     2
25     9
26     3
27     9
28     7
29     1
30    10
31     1
32    10
33     2
34     5
35     0
36     7
37    10
38     4
39     5
40     7
41     5
42     8
43     2
44     4
45     9
46     9
47     6
48     8
49     1
Name: target, dtype: int64

In [8]:
train_df.to_csv('train_set_text.csv')
test_df.to_csv('test_set_text.csv')